In [96]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
 
from sklearn import metrics #метрики
from sklearn import model_selection #методы разделения и валидации
from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
# plt.style.use('seaborn') #стиль отрисовки seaborn
%matplotlib inline

# для нормализации, стандартизации
from sklearn import preprocessing

import category_encoders as ce # импорт для работы с кодировщиком
from sklearn import preprocessing

In [97]:
df = pd.read_csv('D:\\main\\1_projects\educational_projects\\project_kaggle_car_price\\data\\train_data.csv')

Работа с пропусками

In [98]:
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = df.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
df = df.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
df = df.dropna(how='any', axis=0)
df = df.drop('row_ID', axis=1)
#отображаем результирующую долю пропусков
print(df.isnull().mean())
print(df.shape)


vehicle_manufacturer      0.0
vehicle_model             0.0
vehicle_category          0.0
current_mileage           0.0
vehicle_year              0.0
vehicle_gearbox_type      0.0
doors_cnt                 0.0
wheels                    0.0
vehicle_color             0.0
vehicle_interior_color    0.0
car_leather_interior      0.0
deal_type                 0.0
final_price               0.0
dtype: float64
(28118, 13)


небольшая обработка данных

In [99]:
df['deal_type'] = df['deal_type'].apply(lambda x: 'sale' if 'Sale' in x else 'rent')
df['vehicle_age'] = 2020 - df['vehicle_year']
df['wheels'] = df['wheels'].apply(lambda x: 'left' if 'Left' in x else 'right')
df['doors_cnt'] = df['doors_cnt'].apply(lambda x: x.strip())
df['vehicle_color'] = df['vehicle_color'].apply(lambda x: x.strip())

In [100]:
df.head()

,vehicle_manufacturer,vehicle_model,vehicle_category,current_mileage,vehicle_year,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_leather_interior,deal_type,final_price,vehicle_age
0,TOYOTA,Aqua s,Sedan,133000,2014,Automatic,4/5,right,Silver,Black,0,sale,3650.0,6
1,MERCEDES-BENZ,C 220,Sedan,24500,2010,Manual,4/5,left,Silver,Black,0,sale,6800.0,10
2,HYUNDAI,Veloster,Hatchback,31000,2016,Tiptronic,2/3,left,Silver,Black,1,sale,6300.0,4
3,HYUNDAI,Santa FE,Jeep,115459,2015,Automatic,4/5,left,Blue,Black,1,sale,14488.0,5
5,MITSUBISHI,Delica,Jeep,149000,2003,Automatic,4/5,right,Silver,Black,0,sale,20.0,17


Попробуем проанализировать данные по самой популярной марке авто

In [101]:
pop_manufacturer = df['vehicle_manufacturer'].value_counts().index[0]
# 'TOYOTA'

In [102]:
mask_manu = df['vehicle_manufacturer'] == 'TOYOTA'
mask_deal = df['deal_type'] == 'sale'

toyota_df = df[mask_manu & mask_deal].drop(['vehicle_manufacturer', 'deal_type', 'vehicle_year'], axis=1)
toyota_df.head()

,vehicle_model,vehicle_category,current_mileage,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_leather_interior,final_price,vehicle_age
0,Aqua s,Sedan,133000,Automatic,4/5,right,Silver,Black,0,3650.0,6
8,Aqua,Hatchback,86658,Automatic,4/5,left,White,Black,1,4870.0,3
26,Camry,Sedan,130478,Automatic,4/5,left,White,Black,1,275.0,6
31,Camry,Sedan,314373,Automatic,4/5,left,Grey,Black,1,125.0,8
46,Camry,Sedan,109798,Automatic,4/5,left,White,Black,1,1750.0,6


Базовая модель

In [103]:
X = toyota_df.drop('final_price', axis=1).select_dtypes(exclude='object')
y = toyota_df['final_price']
# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

print('Train shape: {}'.format(X_train.shape))
print('Valid shape: {}'.format(X_valid.shape))

# обучаем модель линейной регресии
model = linear_model.LinearRegression(
    
).fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)

print(f'Linear reg model:')
print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))

Train shape: (4876, 3)
Valid shape: (1220, 3)
Linear reg model:
MAE score: 2895.245 thou. $
RMSE score: 4241.015 thou. $
MAPE score: 1131.938 %
R2 score: 0.126


In [104]:
#Инициализируем стандартизатор StandardScaler
scaler = preprocessing.StandardScaler()
#Производим стандартизацию
X_scaled = scaler.fit_transform(X)
#Составляем DataFrame из результата
features = X.columns
X_scaled = pd.DataFrame(X_scaled, columns=features)

# разделяем на тренировочную и тестовую выборки
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# обучаем модель SGD
model = linear_model.SGDRegressor(
    random_state=42,
    
).fit(X_train, y_train)

# делаем предсказание
y_train_predict = model.predict(X_train)

print(f'SGD model:')
print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y_train, y_train_predict)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_predict))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y_train, y_train_predict) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y_train, y_train_predict)))


SGD model:
MAE score: 2911.089 thou. $
RMSE score: 4245.129 thou. $
MAPE score: 1122.159 %
R2 score: 0.124


Избавимся от выбросов

In [105]:
def outliers_z_score(data, feature, log_scale=False, lower_b=1, upper_b=1):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = (mu - 3 * sigma)*lower_b
    upper_bound = (mu + 3 * sigma)*upper_b
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

In [114]:
print(toyota_df.shape)
outliers, cleaned = outliers_z_score(toyota_df, 'current_mileage', log_scale=True, lower_b=0)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')
toyota_df = toyota_df.drop(outliers.index, axis=0)

outliers, cleaned = outliers_z_score(toyota_df, 'final_price', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')
toyota_df = toyota_df.drop(outliers.index, axis=0)

outliers, cleaned = outliers_z_score(toyota_df, 'vehicle_age', log_scale=True, lower_b=0)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')
toyota_df = toyota_df.drop(outliers.index, axis=0)

(6068, 11)
Число выбросов по методу z-отклонения: 0
Результирующее число записей: 5873
Число выбросов по методу z-отклонения: 0
Результирующее число записей: 6068
Число выбросов по методу z-отклонения: 0
Результирующее число записей: 6053


In [72]:
toyota_df

,vehicle_model,vehicle_category,current_mileage,vehicle_gearbox_type,doors_cnt,wheels,vehicle_color,vehicle_interior_color,car_leather_interior,final_price,vehicle_age
0,Aqua s,Sedan,133000,Automatic,4/5,right,Silver,Black,0,3650.0,6
8,Aqua,Hatchback,86658,Automatic,4/5,left,White,Black,1,4870.0,3
26,Camry,Sedan,130478,Automatic,4/5,left,White,Black,1,275.0,6
31,Camry,Sedan,314373,Automatic,4/5,left,Grey,Black,1,125.0,8
46,Camry,Sedan,109798,Automatic,4/5,left,White,Black,1,1750.0,6
...,...,...,...,...,...,...,...,...,...,...,...
34963,Avalon,Sedan,51000,Automatic,4/5,left,White,Black,1,14900.0,5
34978,Camry,Sedan,115312,Automatic,4/5,left,Grey,Black,1,225.0,6
34979,Camry,Sedan,254230,Automatic,4/5,left,Black,Black,1,350.0,6
34993,Avalon limited,Sedan,132000,Automatic,4/5,left,Carnelian red,White,1,10000.0,7
